In [8]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Create engine: engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

**Q10:** Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [41]:
sql_query = ""
sql_query += "SELECT memberrevenue.name as name, (guestrevenue.revenue + memberrevenue.revenue) as revenue "
sql_query += "FROM (SELECT f.name, (f.guestcost * bg.slots) as revenue "
sql_query +=       "FROM Facilities as f,  "
sql_query +=           "(SELECT facid, sum(slots) as slots "
sql_query +=            "FROM Bookings "
sql_query +=            "WHERE memid = 0 "
sql_query +=            "GROUP BY facid) as bg "
sql_query +=       "WHERE f.facid = bg.facid) as guestrevenue, "
sql_query +=      "(SELECT f.name, (f.membercost * bm.slots) as revenue "
sql_query +=       "FROM Facilities as f,  "
sql_query +=           "(SELECT facid, sum(slots) as slots "
sql_query +=            "FROM Bookings "
sql_query +=            "WHERE memid != 0 "
sql_query +=            "GROUP BY facid) as bm "
sql_query +=       "WHERE f.facid = bm.facid) as memberrevenue "
sql_query += "WHERE guestrevenue.name = memberrevenue.name "
sql_query += "AND (guestrevenue.revenue + memberrevenue.revenue) < 1000 "
sql_query += "ORDER BY revenue "
print(sql_query)

SELECT memberrevenue.name as name, (guestrevenue.revenue + memberrevenue.revenue) as revenue FROM (SELECT f.name, (f.guestcost * bg.slots) as revenue FROM Facilities as f,  (SELECT facid, sum(slots) as slots FROM Bookings WHERE memid = 0 GROUP BY facid) as bg WHERE f.facid = bg.facid) as guestrevenue, (SELECT f.name, (f.membercost * bm.slots) as revenue FROM Facilities as f,  (SELECT facid, sum(slots) as slots FROM Bookings WHERE memid != 0 GROUP BY facid) as bm WHERE f.facid = bm.facid) as memberrevenue WHERE guestrevenue.name = memberrevenue.name AND (guestrevenue.revenue + memberrevenue.revenue) < 1000 ORDER BY revenue 


In [42]:
with engine.connect() as con:
    rs = con.execute(text(sql_query))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


A second query for the same question was created here:

In [39]:
sql_query = ""
sql_query += "SELECT f.name, (f.membercost * b.memberslots + f.guestcost * b.guestslots) as revenue "
sql_query += "FROM Facilities as f,  "
sql_query +=     "(SELECT facid,  "
sql_query +=            "(SELECT sum(slots) FROM Bookings WHERE facid = inb.facid and memid != 0) AS memberslots, "
sql_query +=            "(SELECT sum(slots) FROM Bookings WHERE facid = inb.facid and memid = 0) AS guestslots "
sql_query +=      "FROM Bookings as inb "
sql_query +=      "GROUP BY facid) as b "
sql_query += "WHERE f.facid = b.facid "
sql_query += "AND (f.membercost * b.memberslots + f.guestcost * b.guestslots) < 1000 "
sql_query += "ORDER BY revenue "
print(sql_query)

SELECT f.name, (f.membercost * b.memberslots + f.guestcost * b.guestslots) as revenue FROM Facilities as f,  (SELECT facid,  (SELECT sum(slots) FROM Bookings WHERE facid = inb.facid and memid != 0) AS memberslots, (SELECT sum(slots) FROM Bookings WHERE facid = inb.facid and memid = 0) AS guestslots FROM Bookings as inb GROUP BY facid) as b WHERE f.facid = b.facid AND (f.membercost * b.memberslots + f.guestcost * b.guestslots) < 1000 ORDER BY revenue 


In [40]:
with engine.connect() as con:
    rs = con.execute(text(sql_query))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


**Q11:** Produce a report of members and who recommended them in alphabetic surname,firstname order

In [13]:
sql_query = ""
sql_query += "SELECT m.surname as member_surname, m.firstname as member_firstname, r.surname as recommended_surname, r.firstname as recommended_firstname "
sql_query += "FROM Members m "
sql_query += "LEFT JOIN Members r "
sql_query += "ON r.memid = m.recommendedby "
sql_query += "WHERE m.memid != 0 "
sql_query += "ORDER BY m.surname, m.firstname"
print(sql_query)

SELECT m.surname as member_surname, m.firstname as member_firstname, r.surname as recommended_surname, r.firstname as recommended_firstname FROM Members m LEFT JOIN Members r ON r.memid = m.recommendedby WHERE m.memid != 0 ORDER BY m.surname, m.firstname


In [14]:
with engine.connect() as con:
    rs = con.execute(text(sql_query))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df)

       member_surname member_firstname recommended_surname  \
0               Bader         Florence            Stibbons   
1               Baker             Anne            Stibbons   
2               Baker          Timothy             Farrell   
3              Boothe              Tim              Rownam   
4             Butters           Gerald               Smith   
5              Coplin             Joan               Baker   
6             Crumpet            Erica               Smith   
7                Dare            Nancy            Joplette   
8             Farrell            David                None   
9             Farrell           Jemima                None   
10            Genting          Matthew             Butters   
11               Hunt             John             Purview   
12              Jones            David            Joplette   
13              Jones          Douglas               Jones   
14           Joplette           Janice               Smith   
15      

**Q12:** Find the facilities with their usage by member, but not guests

In [15]:
sql_query = ""
sql_query += "SELECT m.surname, m.firstname, f.name as facility_name, count(b.facid) as times_used "
sql_query += "FROM Bookings as b "
sql_query += "LEFT JOIN Facilities as f "
sql_query += "ON b.facid = f.facid "
sql_query += "LEFT JOIN Members as m "
sql_query += "ON b.memid = m.memid "
sql_query += "WHERE b.memid != 0 "
sql_query += "GROUP BY b.memid, b.facid "
sql_query += "ORDER BY b.memid, f.name"
print(sql_query)

SELECT m.surname, m.firstname, f.name as facility_name, count(b.facid) as times_used FROM Bookings as b LEFT JOIN Facilities as f ON b.facid = f.facid LEFT JOIN Members as m ON b.memid = m.memid WHERE b.memid != 0 GROUP BY b.memid, b.facid ORDER BY b.memid, f.name


In [16]:
with engine.connect() as con:
    rs = con.execute(text(sql_query))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df)

     surname firstname    facility_name  times_used
0      Smith    Darren  Badminton Court         132
1      Smith    Darren   Massage Room 1          28
2      Smith    Darren       Pool Table          28
3      Smith    Darren    Snooker Table          12
4      Smith    Darren     Squash Court          14
..       ...       ...              ...         ...
197     Hunt      John   Tennis Court 2           4
198  Crumpet     Erica  Badminton Court           2
199  Crumpet     Erica   Massage Room 1           2
200  Crumpet     Erica     Table Tennis           2
201  Crumpet     Erica   Tennis Court 1           1

[202 rows x 4 columns]


**Q13:** Find the facilities usage by month, but not guests

In [31]:
sql_query = ""
sql_query += "SELECT f.name, count(b.facid) as times_used, strftime('%m', date(starttime)) as month_of_year "
sql_query += "FROM Bookings as b "
sql_query += "LEFT JOIN Facilities as f "
sql_query += "ON b.facid = f.facid "
sql_query += "WHERE b.memid != 0 "
sql_query += "GROUP BY b.facid, strftime('%m', date(starttime)) "
sql_query += "ORDER BY month_of_year, f.name"
print(sql_query)

SELECT f.name, count(b.facid) as times_used, strftime('%m', date(starttime)) as month_of_year FROM Bookings as b LEFT JOIN Facilities as f ON b.facid = f.facid WHERE b.memid != 0 GROUP BY b.facid, strftime('%m', date(starttime)) ORDER BY month_of_year, f.name


In [32]:
with engine.connect() as con:
    rs = con.execute(text(sql_query))
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df)

               name  times_used month_of_year
0   Badminton Court          51            07
1    Massage Room 1          77            07
2    Massage Room 2           4            07
3        Pool Table         103            07
4     Snooker Table          68            07
5      Squash Court          23            07
6      Table Tennis          48            07
7    Tennis Court 1          65            07
8    Tennis Court 2          41            07
9   Badminton Court         132            08
10   Massage Room 1         153            08
11   Massage Room 2           9            08
12       Pool Table         272            08
13    Snooker Table         154            08
14     Squash Court          85            08
15     Table Tennis         143            08
16   Tennis Court 1         111            08
17   Tennis Court 2         109            08
18  Badminton Court         161            09
19   Massage Room 1         191            09
20   Massage Room 2          14   